In [2]:
## Session을 이용해 로그인하기
## Request의 Session
import requests

# Session 생성
s = requests.Session()


In [9]:
## 이렇게 만들어진 세션은 parser.py 에서 requests 위치를 대신한다

# parser.py
import requests

# Session 생성
s = requests.Session()

# HTTP GET Request: requests대신 s 객체를 사용
req = s.get('https://www.clien.net/service/')

# HTML 소스 가져오기
html = req.text

# HTTP Header 가져오기
header = req.headers

# HTTP status 가져오기 (200: 정상)
status = req.status_code

# HTTP가 정상적으로 되었는지 (True/False)
is_ok = req.ok

In [4]:
## 위 코드와 아래코드는 정확히 동일하게 동작하지만
## 위쪽 코드의 경우 Session이 가끔 풀리는 경우가 있어 아래코드로 진행하는것을 추천

In [5]:
# parser.py
import requests

with requests.Session() as s:
    # HTTP GET Request: requests 대신 s 객체를 사용한다.
    req = s.get('https://www.clien.net/service/')
    # HTML 소스 가져오기
    html = req.text
    # HTTP Header 가져오기
    header = req.headers
    # HTTP Status 가져오기 (200: 정상)
    status = req.status_code
    # HTTP가 정상적으로 되었는지 (True/False)
    is_ok = req.ok

In [14]:
# parser.py
import requests

# 로그인할 유저정보를 넣어주자 (모두 문자열)
LOGIN_INFO = {
    'userId': 'taske92',
    'userPassword': 'ehdrbs92'
}

# Session 생성, with 구문 안에서 유지
with requests.Session() as s:
    # HTTP POST Request: 로그인을 위해 POST url와 함께 전송될 data를 넣어주자.
    login_req = s.post('https://www.clien.net/service/login', data=LOGIN_INFO)
    # 어떤 결과가 나올까요?
    print(login_req.status_code)

200


In [28]:
# parser.py
import requests
from bs4 import BeautifulSoup as bs

# 로그인할 유저정보를 넣어주자 (모두 문자열)
LOGIN_INFO = {
    'userId': 'taske92',
    'userPassword': 'ehdrbs92'
}

# Session 생성, with 구문 안에서 유지
with requests.Session() as s:
    # 우선 클리앙 홈페이지에 들어가 봅시다.
    first_page = s.get('https://www.clien.net/service')
    html = first_page.text
    soup = bs(html, 'html.parser')
    csrf = soup.find('input', {'name': '_csrf'}) # input 태그 중에서 name 이 _crsf인 것을 찾습니다.
    print(csrf['value']) # 위에서 찾은 태그의 value를 가져옵니다.
    
    # 이제 LOGIN_INFO에 crsf값을 넣어줍시다.
    # (p.s.)Python3에서 두 dict를 합치는 방법은 {**dict1, **dict2} 으로 dict들을 unpacking하는 것입니다.
    LOGIN_INFO = {**LOGIN_INFO, **{'_csrf': csrf['value']}}
    print(LOGIN_INFO)
    
    # 이제 다시 로그인을 해봅시다.
    login_req = s.post('https://www.clien.net/service/login', data=LOGIN_INFO)
    # 어떤 결과가 나올까요? (200이면 성공!)
    print(login_req.status_code)

abfc8712-0c5c-4274-a8b9-777641bee589
{'userId': 'taske92', 'userPassword': 'ehdrbs92', '_csrf': 'abfc8712-0c5c-4274-a8b9-777641bee589'}
200


## 진짜 데이터를 가져와봅시다
> 이제 로그인이 실패한 경우 Exception을 만들고, 성공일 경웅에는 회원 장터의 게시글을 가져와봅시다.



In [38]:
# parser.py
import requests
from bs4 import BeautifulSoup as bs

# 로그인할 유저정보를 넣어줍시다. (모두 문자열입니다!)
LOGIN_INFO = {
    "userId": 'taske',
    "userPassword": 'ehdrbs92'
}

# Session 생성, with 구문 안에서 유지
with requests.Session() as s:
    # 우선 클리앙 홈페이지에 들어가 봅시다.
    first_page = s.get("https://www.clien.net/service")
    html = first_page.text
    soup = bs(html, 'html.parser')
    csrf = soup.find('input', {'name': '_csrf'}) # input 태그중에 name이 _csrf인 것을 찾는다
    print(csrf['value']) # 위에서 찾은 태그의 value를 가져옵니다.
    
    # 이제 LOGIN_INFO에 csrf값을 넣어줍시다.
    # (p.s.)Python3에서 두 dict를 합치는 방법은 {**dict1, **dict2} 으로 dict들을 unpacking하는 것입니다.
    LOGIN_INFO = {**LOGIN_INFO, **{'_csrf': csrf['value']}}
    print(LOGIN_INFO)
    
    # 이제 다시 로그인을 해봅시다.
    login_req = s.post("https://www.clien.net/service", data=LOGIN_INFO)
    # 어떤 결과가 나올까요? (200이면 성공!)
    print(login_req.status_code)
    # 로그인이 되지 않으면 경고를 띄워줍시다.
    if login_req.status_code != 200:
        raise Exception('로그인이 되지 않았어요! 아이디와 비밀번호를 다시한번 확인해 주세요')
        
    # -- 여기서부터는 로그인이 된 세션이 유지됩니다 --
    # 이제 장터의 게시글 하나를 가져와 봅시다. 아래 예제 링크는 중고장터 공지글입니다.
    post_one = s.get('https://www.clien.net/service/board/rule/10707408')
    soup = bs(post_one.text, 'html.parser')
    # 아래 CSS Selector는 공지글 제목을 콕 하고 집어줍니다.
    title = soup.select('#div_content > div.post_title.symph_row > h3 > span')
    content = soup.select('#div_content > div.post_view')
    # HTML을 제대로 파싱한 뒤에는 .text속성을 이용합니다.
    print(title[0].text) # 글제목의 문자만을 가져와봅시다.
    # [0] 을하는 이유는 select로 하나만 가져와도 title자체는 리스트이기 때문입니다.
    # 즉, 제목 글자는 title이라는 리스트의 0번(첫번째)에 들어가 있습니다.
    print(content[0].text) # 글내용도 마찬가지겟지요?

cdf9aa66-1aa1-4c25-a66e-280c0b781fed
{'userId': 'taske', 'userPassword': 'ehdrbs92', '_csrf': 'cdf9aa66-1aa1-4c25-a66e-280c0b781fed'}
200
회원중고장터 게시판 이용규칙



 2017-05-07 02:12:05
							 
								수정일 : 2020-11-19 13:21:52

 182.♡.24.14







 회원중고장터 게시판은 회원간의 중고 제품 거래를 위한 게시판입니다.
거래시 직거래 또는 안전거래사이트( http://www.unicro.co.kr 등)를 이용하여 장터사기를 미연에 방지하시기 바랍니다.
거래전 사기피해 정보사이트( http://www.thecheat.co.kr )를 검색하여주시기 바랍니다.

회원중고장터는 

가입일 : 200일 이상
로그인 : 80회 이상
게시물수 : 20개 이상
댓글수 : 100개 이상

을 만족하여야 거래게시물을 등록하실 수 있습니다.

0. 사이트 이용규칙을 준수합니다.

1. 게시물 등록시 아래 규칙을 준수합니다.
(1) 전문 업자의 활동을 금지합니다.

전문적인 판매 또는 구매를 위한 활동이 확인되는 경우 게시물은 삭제되며, 작성자는 이용이 제한될 수 있습니다.

(2) 장터 양식을 빠짐없이 입력해야 합니다.

기재사항을 누락하거나, 양식을 임의로 변경해서는 안됩니다.
거래자정보가 입력되지 않거나 가격이 입력되지 않는 게시물 또는 경매성 게시물은 올릴 수 없습니다.
모바일 기기 등으로 게시물 등록할 경우도 양식은 반드시 입력되어야 합니다.

(3) 대리구매 방식으로는 거래하실 수 없습니다.

다만, 영화/공연 등의 대리예매는 거래사고 가능성이 상대적으로 낮고, 회원님들의 요청이 많아 임시 허용됩니다.

(4) 시세보다 비싸다는 댓글, 판매가격보다 싼 물품의 링크 등 불필요한 가격시비를 하지 않습니다.
2. 아래의 상품은 관련법에 의해 거래가 금지됩니다.

군용품 (총포류, 대검류)
이미테이션